In [59]:
import pandas as pd
top50_df = pd.read_csv("top10s_2010_2019.csv", encoding='latin-1')
top50_df.head()

top50_df = top50_df.rename(columns={"pop" : "Popularity"})

In [60]:
import numpy as np
from pathlib import Path
from collections import Counter

In [61]:
#Remove uneccessary columns
top50_df.drop(['Unnamed: 0', 'title', 'artist', 'year'],  inplace=True, axis=1)
top50_df.head()

,top genre,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,Popularity
0,neo mellow,97,89,67,-4,8,80,217,19,4,83
1,detroit hip hop,87,93,75,-5,52,64,263,24,23,82
2,dance pop,120,84,76,-3,29,71,200,10,14,80
3,dance pop,119,92,70,-4,8,71,295,0,4,79
4,pop,109,84,64,-5,9,43,221,2,4,78


In [62]:
top50_df.describe()

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,Popularity
count,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000,603.000000
mean,118.545605,70.504146,64.379768,-5.578773,17.774461,52.225539,224.674959,14.326700,8.358209,66.520730
std,24.795358,16.310664,13.378718,2.798020,13.102543,22.513020,34.130059,20.766165,7.483162,14.517746
min,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,134.000000,0.000000,0.000000,0.000000
25%,100.000000,61.000000,57.000000,-6.000000,9.000000,35.000000,202.000000,2.000000,4.000000,60.000000
50%,120.000000,74.000000,66.000000,-5.000000,12.000000,52.000000,221.000000,6.000000,5.000000,69.000000
75%,129.000000,82.000000,73.000000,-4.000000,24.000000,69.000000,239.500000,17.000000,9.000000,76.000000
max,206.000000,98.000000,97.000000,-2.000000,74.000000,98.000000,424.000000,99.000000,48.000000,99.000000


In [63]:
#Get the dummies and store it in a variable
dummies = pd.get_dummies(top50_df["top genre"])
#Concatenate the dummies to the original dataframe
#top50_df = pd.concat([top50_df, dummies], axis='columns')
#Drop the values
#top50_df.drop(['top genre'], inplace=True, axis='columns')
#Print new dataframe
#top50_df.head()

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,Popularity,...,house,indie pop,irish singer-songwriter,latin,metropopolis,moroccan pop,neo mellow,permanent wave,pop,tropical house
0,97,89,67,-4,8,80,217,19,4,83,...,0,0,0,0,0,0,1,0,0,0
1,87,93,75,-5,52,64,263,24,23,82,...,0,0,0,0,0,0,0,0,0,0
2,120,84,76,-3,29,71,200,10,14,80,...,0,0,0,0,0,0,0,0,0,0
3,119,92,70,-4,8,71,295,0,4,79,...,0,0,0,0,0,0,0,0,0,0
4,109,84,64,-5,9,43,221,2,4,78,...,0,0,0,0,0,0,0,0,1,0


In [64]:
##Can't get it to add this column...

#Seperating Popular songs to >85 (Y) and <=84 (N (less popular))    
top50_df['Pop Y/N'] = pd.Series(np.where(top50_df.Popularity.values <= 80, 0, 1), top50_df.index)
top50_df.head()

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,Popularity,...,indie pop,irish singer-songwriter,latin,metropopolis,moroccan pop,neo mellow,permanent wave,pop,tropical house,Pop Y/N
0,97,89,67,-4,8,80,217,19,4,83,...,0,0,0,0,0,1,0,0,0,1
1,87,93,75,-5,52,64,263,24,23,82,...,0,0,0,0,0,0,0,0,0,1
2,120,84,76,-3,29,71,200,10,14,80,...,0,0,0,0,0,0,0,0,0,0
3,119,92,70,-4,8,71,295,0,4,79,...,0,0,0,0,0,0,0,0,0,0
4,109,84,64,-5,9,43,221,2,4,78,...,0,0,0,0,0,0,0,1,0,0


In [65]:
#Drop 'Popularity' Column
top50_df.drop("Popularity", axis=1, inplace=True)
top50_df.head()

,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,acoustic pop,...,indie pop,irish singer-songwriter,latin,metropopolis,moroccan pop,neo mellow,permanent wave,pop,tropical house,Pop Y/N
0,97,89,67,-4,8,80,217,19,4,0,...,0,0,0,0,0,1,0,0,0,1
1,87,93,75,-5,52,64,263,24,23,0,...,0,0,0,0,0,0,0,0,0,1
2,120,84,76,-3,29,71,200,10,14,0,...,0,0,0,0,0,0,0,0,0,0
3,119,92,70,-4,8,71,295,0,4,0,...,0,0,0,0,0,0,0,0,0,0
4,109,84,64,-5,9,43,221,2,4,0,...,0,0,0,0,0,0,0,1,0,0


In [66]:
X = top50_df.drop("Pop Y/N", axis=1)

y = top50_df["Pop Y/N"]

In [67]:
#balance of 'Pop Y/N' values
y.value_counts()

0    534
1     69
Name: Pop Y/N, dtype: int64

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Naive Random Oversampling

In [69]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state = 1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

#View the Count
Counter(y_resampled)

Counter({0: 401, 1: 401})

In [70]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver="lbfgs", random_state =1)
model.fit(X_resampled, y_resampled)

C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [71]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(y_test, y_pred))

0.6146616541353384


In [72]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[97, 36],
       [ 9,  9]], dtype=int64)

In [73]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.92      0.73      0.50      0.81      0.60      0.37       133
          1       0.20      0.50      0.73      0.29      0.60      0.36        18

avg / total       0.83      0.70      0.53      0.75      0.60      0.37       151



# SMOTE Oversampling

In [74]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({0: 401, 1: 401})

In [75]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

C:\Users\mleegan\Anaconda3\envs\mlenv\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(random_state=1)

In [76]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6829573934837092

In [77]:
##Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[93, 40],
       [ 6, 12]], dtype=int64)

In [78]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.94      0.70      0.67      0.80      0.68      0.47       133
          1       0.23      0.67      0.70      0.34      0.68      0.46        18

avg / total       0.85      0.70      0.67      0.75      0.68      0.47       151



# Balanced Random Forest Classifier

In [79]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
rf_model.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [80]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
print(balanced_accuracy_score(y_test, y_pred))

0.6528822055137844


In [81]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[85, 48],
       [ 6, 12]], dtype=int64)

In [82]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.93      0.64      0.67      0.76      0.65      0.42       133
          1       0.20      0.67      0.64      0.31      0.65      0.43        18

avg / total       0.85      0.64      0.66      0.71      0.65      0.43       151



In [83]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(rf_model.feature_importances_, feature_names), reverse=True)

[(0.12146474418232746, 'nrgy'),
 (0.11221692305507794, 'acous'),
 (0.10731372108752475, 'bpm'),
 (0.10680437221182887, 'dnce'),
 (0.10247381572940574, 'dur'),
 (0.09054267136666769, 'val'),
 (0.07978400962015682, 'live'),
 (0.07345407723031416, 'spch'),
 (0.05697200168160849, 'dB'),
 (0.038980464068584915, 'dance pop'),
 (0.01846743071952176, 'pop'),
 (0.011510173240675365, 'canadian pop'),
 (0.010282948082914651, 'canadian contemporary r&b'),
 (0.008506259228309202, 'big room'),
 (0.008136078312403074, 'electropop'),
 (0.00719809216356866, 'brostep'),
 (0.007196975738892612, 'boy band'),
 (0.003088414514370616, 'detroit hip hop'),
 (0.0029460920449499604, 'escape room'),
 (0.0029358829902903475, 'electronic trap'),
 (0.00279954052457632, 'neo mellow'),
 (0.0024596802253546565, 'edm'),
 (0.00229185183324752, 'barbadian pop'),
 (0.0021371911785776107, 'british soul'),
 (0.0018717298920275058, 'australian dance'),
 (0.0016108121985718138, 'irish singer-songwriter'),
 (0.00147772084392342